In [1]:
import numpy as np
import pandas as pd

In [2]:
ratings=pd.read_csv(r"E:\PYTHON\python Datasets\ratings.csv")

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
movies=pd.read_csv(r"E:\PYTHON\python Datasets\movies.csv")

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
#no of unique users
n_users=ratings.userId.nunique()
n_users

7120

In [7]:
#no of movies
n_users=movies.movieId.nunique()
n_users

27278

In [8]:
#pivot and filling null with 0
rating=ratings.pivot(index='userId',columns='movieId',values='rating').fillna(0)
rating.shape

(7120, 14026)

In [9]:
rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm.
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8468  0.8457  0.8428  0.8451  0.0017  
MAE (testset)     0.6487  0.6484  0.6458  0.6476  0.0013  
Fit time          74.57   71.68   70.86   72.37   1.59    
Test time         6.29    6.23    6.75    6.42    0.23    


{'test_rmse': array([0.8467636 , 0.84569112, 0.84283734]),
 'test_mae': array([0.64873497, 0.64836143, 0.64584506]),
 'fit_time': (74.57023501396179, 71.68354845046997, 70.86439824104309),
 'test_time': (6.290048360824585, 6.234004497528076, 6.747944116592407)}

In [11]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


#### To find all the movies rated as more than 4 stars by user with userId = 1

In [26]:
ratings_1 = ratings[(ratings['userId'] == 1) & (ratings['rating'] >= 4)]

# Set the index of the ratings_1 DataFrame to be the 'movieId' column
ratings_1 = ratings_1.set_index('movieId')

#joining with movies table and selecting title column
ratings_1 = ratings_1.join(movies).title
ratings_1.head(10)

movieId
151                                 Batman Forever (1995)
223                                      Dream Man (1995)
253                                         Junior (1994)
260                              Ladybird Ladybird (1994)
293                                   Pulp Fiction (1994)
296                                         Priest (1994)
318     Strawberry and Chocolate (Fresa y chocolate) (...
541                                          Harem (1985)
1036                                          Jude (1996)
1079                                       Top Gun (1986)
Name: title, dtype: object

In [27]:
user_1 = movies.copy()

#Reset the index for user_1 dataset
user_1 = user_1.reset_index()

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


#create a training set for svd
trainset = data.build_full_trainset()
svd.fit(trainset)

#Predict the ratings for user1
user_1['Estimate_Score'] = user_1['movieId'].apply(lambda x: svd.predict(1, x).est)

#Drop extra columns from the user1 data frame
user_1 = user_1.drop(['movieId','genres','index'], axis = 1)

# Sort predicted ratings for user1 in descending order
user_1 = user_1.sort_values('Estimate_Score', ascending=False)

print(user_1.head(10))

                                                   title  Estimate_Score
5853       Lord of the Rings: The Two Towers, The (2002)        4.831795
4897   Lord of the Rings: The Fellowship of the Ring,...        4.807515
7041   Lord of the Rings: The Return of the King, The...        4.795391
9250                            10th Kingdom, The (2000)        4.585090
6873   Passion of Joan of Arc, The (Passion de Jeanne...        4.535947
7215                                      Persona (1966)        4.495814
9448                     Head-On (Gegen die Wand) (2004)        4.494157
7973   Short Film About Killing, A (Krótki film o zab...        4.493435
18528                                Intouchables (2011)        4.484848
3003                                      Matewan (1987)        4.484285


In [ ]:
#this is the movies recomandation for user 1